In [1]:
import pandas as pd
import requests
from pathlib import Path
from datetime import datetime, timezone
import re

In [2]:
pd.set_option("display.max_colwidth", 1000)

In [3]:
COURSE_ID = "iteration_4"

In [9]:
df_hw_links = pd.read_csv(f"/usr/local/share/nbgrader/exchange/{COURSE_ID}/hw_part_2.csv")
df_hw_links

,Фамилия и имя,Номер ДЗ,Вложения
0,Дистлер Марина,Домашнее задание №1,hse_metrics_Distler.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/26/e790ead958ee4221aca5bd0b1c85259c/hse_metrics.ipynb)
1,Ишмуратов Сергей,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/27/712cf1b35cde497c9da7c39e39d2602f/hse_metrics.ipynb)
2,Назарько Михаил,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/29/7258e7baaa374585848e50a76e81fcea/hse_metrics.ipynb)
3,Конохова Екатерина,Домашнее задание №1,Конохова.Е.М_ДЗ1.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/30/9cde4e0c462e46818614e66647568a1f/Конохова.Е.М_ДЗ1.ipynb)
4,Шевченко Кирилл,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/03/e5cfbe09979c4fef93b1a752c6a3eb9b/hse_metrics.ipynb)
5,Дельман Александр,Домашнее задание №1,hse_metrics Delman.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/04/5dd727eb654c4982b5c9c2ad6fde1f25/hse_metrics Delman.ipynb)
6,Пеганов Никита,Домашнее задание №1,hse_metrics (1).ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/06/e704c9e0de9d42edbd27070c6987a4b1/hse_metrics (1).ipynb)
7,Козлов Кирилл,Домашнее задание №1,Kozlov-ky_rec.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/9734fa6f46044fe88c6b9c1467206083/Kozlov-ky_rec.ipynb)
8,Партин Максим,Домашнее задание №1,Рек. системы дз1.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/5ee43f6cbd3e48b1bf99b85cdc23cfac/Рек. системы дз1.ipynb)
9,Вольхин Данил,Домашнее задание №1,hse_metrics_done.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/08/c3f27f50569746b8919b5d77ccd0319d/hse_metrics_done.ipynb)


In [10]:
hw_name_map = {
    "Домашнее задание №1": "hw_1",
    "Домашнее задание №2 - Проект": "hw_2",
}
hw_filename_map = {
    "hw_1": "hse_metrics.ipynb",
    "hw_2": "hse_project.ipynb",
}

In [11]:
assert df_hw_links["Номер ДЗ"].isin(hw_name_map.keys()).all()

In [12]:
df_hw_links = df_hw_links[df_hw_links["Вложения"].notna()].copy()

In [13]:
df_hw_links["hw"] = df_hw_links["Номер ДЗ"].map(hw_name_map)
df_hw_links["filename"] = df_hw_links["hw"].map(hw_filename_map)
df_hw_links["name"] = df_hw_links["Фамилия и имя"].apply(lambda s: "_".join(s.split()[:2]))
df_hw_links["link"] = df_hw_links["Вложения"].apply(lambda s: re.match(r".+\((https://tabs\.mts\.ru.+)\)$", s).groups()[0])
df_hw_links

,Фамилия и имя,Номер ДЗ,Вложения,hw,filename,name,link
0,Дистлер Марина,Домашнее задание №1,hse_metrics_Distler.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/26/e790ead958ee4221aca5bd0b1c85259c/hse_metrics.ipynb),hw_1,hse_metrics.ipynb,Дистлер_Марина,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/26/e790ead958ee4221aca5bd0b1c85259c/hse_metrics.ipynb
1,Ишмуратов Сергей,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/27/712cf1b35cde497c9da7c39e39d2602f/hse_metrics.ipynb),hw_1,hse_metrics.ipynb,Ишмуратов_Сергей,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/27/712cf1b35cde497c9da7c39e39d2602f/hse_metrics.ipynb
2,Назарько Михаил,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/29/7258e7baaa374585848e50a76e81fcea/hse_metrics.ipynb),hw_1,hse_metrics.ipynb,Назарько_Михаил,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/29/7258e7baaa374585848e50a76e81fcea/hse_metrics.ipynb
3,Конохова Екатерина,Домашнее задание №1,Конохова.Е.М_ДЗ1.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/30/9cde4e0c462e46818614e66647568a1f/Конохова.Е.М_ДЗ1.ipynb),hw_1,hse_metrics.ipynb,Конохова_Екатерина,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/09/30/9cde4e0c462e46818614e66647568a1f/Конохова.Е.М_ДЗ1.ipynb
4,Шевченко Кирилл,Домашнее задание №1,hse_metrics.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/03/e5cfbe09979c4fef93b1a752c6a3eb9b/hse_metrics.ipynb),hw_1,hse_metrics.ipynb,Шевченко_Кирилл,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/03/e5cfbe09979c4fef93b1a752c6a3eb9b/hse_metrics.ipynb
5,Дельман Александр,Домашнее задание №1,hse_metrics Delman.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/04/5dd727eb654c4982b5c9c2ad6fde1f25/hse_metrics Delman.ipynb),hw_1,hse_metrics.ipynb,Дельман_Александр,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/04/5dd727eb654c4982b5c9c2ad6fde1f25/hse_metrics Delman.ipynb
6,Пеганов Никита,Домашнее задание №1,hse_metrics (1).ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/06/e704c9e0de9d42edbd27070c6987a4b1/hse_metrics (1).ipynb),hw_1,hse_metrics.ipynb,Пеганов_Никита,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/06/e704c9e0de9d42edbd27070c6987a4b1/hse_metrics (1).ipynb
7,Козлов Кирилл,Домашнее задание №1,Kozlov-ky_rec.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/9734fa6f46044fe88c6b9c1467206083/Kozlov-ky_rec.ipynb),hw_1,hse_metrics.ipynb,Козлов_Кирилл,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/9734fa6f46044fe88c6b9c1467206083/Kozlov-ky_rec.ipynb
8,Партин Максим,Домашнее задание №1,Рек. системы дз1.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/5ee43f6cbd3e48b1bf99b85cdc23cfac/Рек. системы дз1.ipynb),hw_1,hse_metrics.ipynb,Партин_Максим,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/07/5ee43f6cbd3e48b1bf99b85cdc23cfac/Рек. системы дз1.ipynb
9,Вольхин Данил,Домашнее задание №1,hse_metrics_done.ipynb (https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/08/c3f27f50569746b8919b5d77ccd0319d/hse_metrics_done.ipynb),hw_1,hse_metrics.ipynb,Вольхин_Данил,https://tabs.mts.ru/attachment/spacespcccvvlpyvzg/2025/10/08/c3f27f50569746b8919b5d77ccd0319d/hse_metrics_done.ipynb


In [14]:
HW_DIR = f"/usr/local/share/nbgrader/exchange/{COURSE_ID}/inbound"

In [15]:
try:
    errored_rows
except NameError:
    errored_rows = []

for _, row in df_hw_links.iterrows():
    print(row['name'])
    hw_link = row["link"]
    resp = requests.get(hw_link)
    if resp.status_code != 200:
        errored_rows = []
        print(f"Error: {resp} ({resp.text})")
        continue
    
    now_dt = datetime.now(tz=timezone.utc)
    hw_name = row["hw"]
    filename = row["filename"]
    target_folder_name = f"{row['name']}+{hw_name}+{now_dt:%Y-%m-%dT%H:%M:%S} UTC+00"
    target_folder_path = Path(HW_DIR) / target_folder_name
    target_folder_path.mkdir(exist_ok=True, parents=True)

    target_path = target_folder_path / filename
    target_path.write_text(resp.text)

Дистлер_Марина
Ишмуратов_Сергей
Назарько_Михаил
Конохова_Екатерина
Шевченко_Кирилл
Дельман_Александр
Пеганов_Никита
Козлов_Кирилл
Партин_Максим
Вольхин_Данил
Дистлер_Марина
Лю_Юрий
Бураков_Даниил
Ширшов_Константин
Nagaev_Denis
гадиев_миша
Шахов_Станислав
Холичева_Ангелина
Сахаутдинова_Анжелика
Лемешко_Владислав
Аскарбек_Казыбек
Кириенко_Владислав
